举例1

In [15]:
# import dotenv
# from langchain.memory import ConversationTokenBufferMemory
# from langchain_classic.memory import ConversationSummaryMemory
# from langchain_openai import ChatOpenAI
#
# # 关键：补齐 Pydantic forward-ref 需要的类型
# from langchain_core.caches import BaseCache
# from langchain_core.language_models import BaseLanguageModel
# from langchain_core.callbacks import Callbacks  # ✅ 新增：解决 Callbacks 未定义
#
# dotenv.load_dotenv()
#
# llm = ChatOpenAI(model="gpt-4o-mini")
#
# # 让 Pydantic 在你当前环境里能找到所有前向引用的类型
# ConversationTokenBufferMemory.model_rebuild(
#     _types_namespace={
#         **globals(),
#         "BaseCache": BaseCache,
#         "BaseLanguageModel": BaseLanguageModel,
#         "Callbacks": Callbacks,  # ✅ 必须提供
#     }
# )
#
# memory = ConversationTokenBufferMemory(
#     llm=llm,
#     max_token_limit=20
# )
#
# memory.save_context({"input": "你好吗？"}, {"output": "我很好，谢谢！"})
# memory.save_context({"input": "今天天气如何？"}, {"output": "晴天，25度"})
#
# print(memory.load_memory_variables({}))


{'history': 'AI: 晴天，25度'}


# ConversationSummaryMemory的使用

In [18]:
# 举例1
import dotenv
from langchain.memory import ConversationSummaryMemory
from langchain_openai import ChatOpenAI

# 关键：补齐 Pydantic forward-ref 需要的类型
from langchain_core.caches import BaseCache
from langchain_core.language_models import BaseLanguageModel
from langchain_core.callbacks import Callbacks  # ✅ 新增：解决 Callbacks 未定义

dotenv.load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini")

# 让 Pydantic 在你当前环境里能找到所有前向引用的类型
ConversationSummaryMemory.model_rebuild(
    _types_namespace={
        **globals(),
        "BaseCache": BaseCache,
        "BaseLanguageModel": BaseLanguageModel,
        "Callbacks": Callbacks,  # ✅ 必须提供
    }
)

memory = ConversationSummaryMemory(
    llm=llm,
)

memory.save_context({"input": "你好吗？"}, {"output": "我很好，谢谢！"})
memory.save_context({"input": "今天天气如何？"}, {"output": "晴天，25度"})

print(memory.load_memory_variables({}))


{'history': 'The human asks the AI how it is, and the AI responds that it is well, thank you! The human then inquires about the weather, to which the AI replies that it is sunny and 25 degrees.'}


举例2

In [22]:
# 1.导入相关包
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain_openai import ChatOpenAI
# 2.定义ChatMessageHistory对象
llm = ChatOpenAI(model="gpt-4o-mini")
# 3.假设原始消息
history = ChatMessageHistory()
history.add_user_message("你好，你是谁？")
history.add_ai_message("我是AI助手小智")
# 4.初始化ConversationSummaryMemory实例
memory = ConversationSummaryMemory.from_messages(
llm=llm,
#是生成摘要的原材料 保留完整对话供必要时回溯。当新增对话时，LLM需要结合原始历史生成新摘要
chat_memory=history,
)
print(memory.load_memory_variables({}))
memory.save_context(inputs={"human":"我的名字叫小明"},outputs={"AI":"很高兴认识你"})
print(memory.load_memory_variables({}))
memory.save_context(inputs={"human":"我的名字叫小明"},outputs={"AI":"很高兴认识你"})
print(memory.load_memory_variables({}))

{'history': 'The human greets the AI and asks who it is. The AI responds that it is the AI assistant named Xiao Zhi.'}
{'history': 'The human greets the AI and asks who it is. The AI responds that it is the AI assistant named Xiao Zhi. The human introduces himself as Xiao Ming, and the AI expresses pleasure in meeting him.'}
{'history': 'The human greets the AI and asks who it is. The AI responds that it is the AI assistant named Xiao Zhi. The human introduces himself as Xiao Ming, and the AI expresses pleasure in meeting him, saying it is glad to meet Xiao Ming.'}


# ConversationSummaryBufferMemory 混合记忆机制

举例1


In [24]:
# 1.导入相关的包
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI
# 2.定义模型
llm = ChatOpenAI(model="gpt-4o-mini",temperature=0)
# 3.定义ConversationSummaryBufferMemory对象
ConversationSummaryBufferMemory.model_rebuild(
    _types_namespace={
        **globals(),
        "BaseCache": BaseCache,
        "BaseLanguageModel": BaseLanguageModel,
        "Callbacks": Callbacks,  # ✅ 必须提供
    }
)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=40,
    return_messages=True
)
# 4.保存消息
memory.save_context({"input": "你好，我的名字叫小明"}, {"output": "很高兴认识你，小明"})
memory.save_context({"input": "李白是哪个朝代的诗人"}, {"output": "李白是唐朝诗人"})
memory.save_context({"input": "唐宋八大家里有苏轼吗？"}, {"output": "有"})
# 5.读取内容
print(memory.load_memory_variables({}))
print(memory.chat_memory.messages)

{'history': [SystemMessage(content='The human introduces themselves as Xiao Ming. The AI expresses pleasure in meeting Xiao Ming, and the human asks which dynasty the poet Li Bai belongs to.', additional_kwargs={}, response_metadata={}), AIMessage(content='李白是唐朝诗人', additional_kwargs={}, response_metadata={}), HumanMessage(content='唐宋八大家里有苏轼吗？', additional_kwargs={}, response_metadata={}), AIMessage(content='有', additional_kwargs={}, response_metadata={})]}
[AIMessage(content='李白是唐朝诗人', additional_kwargs={}, response_metadata={}), HumanMessage(content='唐宋八大家里有苏轼吗？', additional_kwargs={}, response_metadata={}), AIMessage(content='有', additional_kwargs={}, response_metadata={})]
